In [31]:
pip install tensorflow-macos keras Pillow

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import numpy as np
import tensorflow as tf
import keras
import cv2
import argparse

In [33]:
h5_model_path = './saved_model/model.h5'
tf_lite_model_path = './saved_model/model.tflite'

In [34]:
label_path = './saved_model/labels.txt'

In [35]:
image_path = './test/gl2.jpg'

In [36]:
def load_labels(label_path):
    with open(label_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

In [37]:
# predict from image
def h5_predict(image_path, model_path):
    model = keras.models.load_model(model_path)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (200, 200))
    img = np.reshape(img, [1, 200, 200, 3])
    pred = model.predict(img)
    print(pred)
    
    return pred

h5_predicted = h5_predict(image_path, h5_model_path)

[[-2.4822853   0.28746092  1.9939039   1.2676669  -0.08928843  1.1868728
  -0.65853596]]


In [38]:
# predict from tflite
def tflite_predict(image_path, model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    image = cv2.imread(image_path)
    image = cv2.resize(image, (input_details[0]['shape'][1], input_details[0]['shape'][2]))
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    results = np.squeeze(output_data)

    top_k = results.argsort()[-5:][::-1]
    labels = load_labels(label_path)

    for i in top_k:
        print(labels[i], results[i])
    
    return output_data

    
    
tflite_predicted = tflite_predict(image_path, tf_lite_model_path)

glass 1.9939032
headphones 1.2676667
laptop 1.1868728
bottle 0.28745976
keyboard -0.08928744


In [39]:
labels = []
labels_file = open(label_path, 'r')
for line in labels_file:
    labels.append(line.rstrip())
labels_file.close()
    

In [40]:
print(
    "H5 - This image most likely belongs to {} "
    .format(labels[np.argmax(h5_predicted)], 100 * np.max(h5_predicted))
)
print(
    "TFLITE - This image most likely belongs to {}"
    .format(labels[np.argmax(tflite_predicted)], 100 * np.max(tflite_predicted))
)

H5 - This image most likely belongs to glass 
TFLITE - This image most likely belongs to glass
